# Regsvr32 Anomaly
Detects various anomalies in relation to regsvr32.exe

## Rule Content
```
- title: Regsvr32 Anomaly
  id: 8e2b24c9-4add-46a0-b4bb-0057b4e6187d
  status: experimental
  description: Detects various anomalies in relation to regsvr32.exe
  author: Florian Roth
  references:
  - https://subt0x10.blogspot.de/2017/04/bypass-application-whitelisting-script.html
  tags:
  - attack.t1117
  - attack.defense_evasion
  - attack.execution
  - car.2019-04-002
  - car.2019-04-003
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      Image: '*\regsvr32.exe'
      CommandLine: '*\Temp\\*'
    selection2:
      Image: '*\regsvr32.exe'
      ParentImage: '*\powershell.exe'
    selection3:
      Image: '*\regsvr32.exe'
      ParentImage: '*\cmd.exe'
    selection4:
      Image: '*\regsvr32.exe'
      CommandLine:
      - '*/i:http* scrobj.dll'
      - '*/i:ftp* scrobj.dll'
    selection5:
      Image: '*\wscript.exe'
      ParentImage: '*\regsvr32.exe'
    selection6:
      Image: '*\EXCEL.EXE'
      CommandLine: '*..\..\..\Windows\System32\regsvr32.exe *'
    condition: 1 of them
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\regsvr32.exe AND process_command_line.keyword:*\\Temp\\*) OR (process_path.keyword:*\\regsvr32.exe AND process_parent_path.keyword:*\\powershell.exe) OR (process_path.keyword:*\\regsvr32.exe AND process_parent_path.keyword:*\\cmd.exe) OR (process_path.keyword:*\\regsvr32.exe AND process_command_line.keyword:(*\/i\:http*\ scrobj.dll OR *\/i\:ftp*\ scrobj.dll)) OR (process_path.keyword:*\\wscript.exe AND process_parent_path.keyword:*\\regsvr32.exe) OR (process_path.keyword:*\\EXCEL.EXE AND process_command_line.keyword:*..\\..\\..\\Windows\\System32\\regsvr32.exe\ *))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()